# 要約 
このJupyter Notebookは、Kaggleの「LMSYS - Chatbot Arena 人間による好み予測チャレンジ」に参加するためのモデルを構築するプロセスを示しています。Notebookは、様々なデータ処理と機械学習手法を用いて、与えられたプロンプトに対する異なるチャットボットの応答の好みを予測する問題に取り組んでいます。

### 主な内容と手法

1. **データの読み込みと前処理**:
   - データは`pandas`を用いてCSVファイルから読み込まれ、必要に応じてトレーニングデータの一部をサブセットとして利用します。
   - 特定の文字列処理を行い、プロンプトや応答から余分な文字を取り除く`process`関数が定義されています。

2. **特徴量エンジニアリング**:
   - `Preprocessor`クラスでは、テキスト間のコサイン類似度やジャッカード類似度の計算、n-グラムの生成、改行や引用符のカウント、トークン化など、さまざまなテキスト特徴量を計算します。
   - 各応答とプロンプトの類似性を数値化することに焦点をあて、生成した特徴量を基に予測モデルを訓練します。

3. **モデルの構築と評価**:
   - モデルには`XGBoost`ライブラリを使用し、多クラス分類器を訓練します。
   - `StratifiedKFold`を用いてクロスバリデーションを行い、各フォールドの対数損失（log loss）を評価指標として使用します。

4. **結果の可視化と提出ファイルの作成**:
   - 各特徴量の重要度を計算し、`matplotlib`および`seaborn`を使って重要度のバープロットを作成します。
   - 最後に、テストデータに基づいた予測結果を`submission.csv`としてエクスポートし、コンペティションへの提出を可能にします。

### 使用ライブラリ
- `numpy`, `pandas`: データ操作
- `nltk`: テキスト処理
- `matplotlib`, `seaborn`: データ可視化
- `xgboost`: 機械学習モデル
- `sklearn`: モデル評価とデータ前処理

このNotebookは、機械学習における特徴量エンジニアリングの重要性を示すとともに、フレームワークを利用して有効な予測モデルを構築する一連の流れを具体化したものとなっています。

---


# 用語概説 
以下に、Jupyter Notebookに登場する専門用語で、機械学習・深層学習の初心者がつまずきそうなものの簡単な解説を示します。普段あまり使用されない用語や、特にこのノートブックに特有の概念に焦点を当てています。

1. **TF-IDF (Term Frequency-Inverse Document Frequency)**:
   - テキストデータの重要度を測る指標。特定の単語が文書内でどのくらい頻繁に出てくるか（Term Frequency）と、その単語が文書全体でどれくらいの頻度で出現するか（Inverse Document Frequency）を組み合わせて計算する。テキスト分類や情報検索の際に、多く使われる。

2. **コサイン類似度 (Cosine Similarity)**:
   - 2つのベクトル間の類似度を計算する指標です。コサインの角度を用い、ベクトルの向き（意味的な類似性）を比較する。特にテキストデータの類似性を測定する際に有用。

3. **ジャッカード類似度 (Jaccard Similarity)**:
   - 2つの集合の間の類似度を測る指標で、積集合のサイズを和集合のサイズで割ったもの。異なるテキスト間での共通語の割合を測る際に使われ、単語の重複を確認するのに役立つ。

4. **n-グラム (n-gram)**:
   - 連続するn個の要素（通常は単語や文字）から構成される部分列。言語モデルやテキスト処理のタスクにおいて、文脈依存性を持つ情報を捉えるために使用される。

5. **ストラティファイドKフォールド交差検定 (Stratified K-Fold Cross Validation)**:
   - データセットをK個のサブセットに分割し、各サブセットが元のデータセットのクラス分布を保持するような交差検定の手法。特にクラスの不均衡がある場合に有用。

6. **early stopping (早期停止)**:
   - モデル訓練中にValidationの性能が向上しなくなった場合、訓練を自動的に終了させる手法。過学習を防ぐために使用される。

7. **弱学習器 (Weak Learner)**:
   - パフォーマンスがランダムな予測よりもわずかに優れているが、それ自体が強力ではないモデル。バギングやブースティングのアルゴリズムで多く使用される。

8. **対数損失 (Log Loss)**:
   - 分類問題で用いる評価指標。予測の確率に基づいてモデルの性能を測る。0から無限大までの値を取るが、低いほど良い評価を意味する。

9. **フィーチャーインポータンス (Feature Importance)**:
   - モデルがどの特徴量（フィーチャー）を重視しているかを示す指標。XGBoostなどのツリー系モデルでは、各特徴がモデルの予測にどれほど貢献したかを確認できる。

10. **バイグラム (Bigram) / トライグラム (Trigram)**:
    - それぞれ2つまたは3つの連続する単語からなるn-グラム。例えば、「自然言語処理」というフレーズでは、「自然言語」「言語処理」というバイグラムが得られる。

これらの用語は、初心者には難解であるが、ノートブック内で効果的にデータ処理とモデル構築をするために重要な概念です。

---


<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import gc
import os
import re
import numpy as np
import pandas as pd

import nltk
from nltk.util import ngrams
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
```

</div>
<div class="column-right">

# 日本語訳

```python
import gc
import os
import re
import numpy as np
import pandas as pd

import nltk
from nltk.util import ngrams
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
```

</div>
</details>

In [ ]:
import gc
import os
import re
import numpy as np
import pandas as pd

import nltk
from nltk.util import ngrams
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
class config:
    root = "/kaggle/input/lmsys-chatbot-arena/"
    train_path = os.path.join(root, "train.csv")
    test_path = os.path.join(root, "test.csv")
    sample_submission_path = os.path.join(root, "sample_submission.csv")
    seed = 42
    n_splits = 10
```

</div>
<div class="column-right">

# 日本語訳

```python
class config:
    root = "/kaggle/input/lmsys-chatbot-arena/"
    train_path = os.path.join(root, "train.csv")  # トレーニングデータのパス
    test_path = os.path.join(root, "test.csv")    # テストデータのパス
    sample_submission_path = os.path.join(root, "sample_submission.csv")  # サンプル提出データのパス
    seed = 42  # 乱数シードを42に設定
    n_splits = 10  # クロスバリデーションの分割数を10に設定
```

</div>
</details>

In [ ]:
class config:
    root = "/kaggle/input/lmsys-chatbot-arena/"
    train_path = os.path.join(root, "train.csv")  # トレーニングデータのパス
    test_path = os.path.join(root, "test.csv")    # テストデータのパス
    sample_submission_path = os.path.join(root, "sample_submission.csv")  # サンプル提出データのパス
    seed = 42  # 乱数シードを42に設定
    n_splits = 10  # クロスバリデーションの分割数を10に設定

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train = pd.read_csv(config.train_path)
test = pd.read_csv(config.test_path)
sample_submission = pd.read_csv(config.sample_submission_path)

if test.shape[0] < 10:
    train = train.iloc[:10000]
    
def process(input_str):
    stripped_str = input_str.strip('[]')
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    return  ' '.join(sentences)

train["prompt"] = train["prompt"].apply(process)
train["response_a"] = train["response_a"].apply(process)
train["response_b"] = train["response_b"].apply(process)

test["prompt"] = test["prompt"].apply(process)
test["response_a"] = test["response_a"].apply(process)
test["response_b"] = test["response_b"].apply(process)

print(f"train shape: {train.shape}")
print(f"test shape: {test.shape}")
print("-"*90)
print(f"train missing values: {train.isnull().sum().sum()}")
print(f"test missing values: {test.isnull().sum().sum()}")
print("-"*90)

train.head()
```

</div>
<div class="column-right">

# 日本語訳

```python
train = pd.read_csv(config.train_path)  # トレーニングデータを読み込む
test = pd.read_csv(config.test_path)    # テストデータを読み込む
sample_submission = pd.read_csv(config.sample_submission_path)  # サンプル提出データを読み込む

if test.shape[0] < 10:  # テストデータのサンプル数が10未満の場合
    train = train.iloc[:10000]  # 最初の10000行のみをトレーニングデータに使用
    
def process(input_str):  # 入力文字列を処理する関数
    stripped_str = input_str.strip('[]')  # 角括弧を除去
    sentences = [s.strip('"') for s in stripped_str.split('","')]  # 行を分割し、不要なダブルクオートを除去
    return ' '.join(sentences)  # 処理した文を空白区切りで結合して返す

train["prompt"] = train["prompt"].apply(process)  # "prompt"列を処理
train["response_a"] = train["response_a"].apply(process)  # "response_a"列を処理
train["response_b"] = train["response_b"].apply(process)  # "response_b"列を処理

test["prompt"] = test["prompt"].apply(process)  # テストデータの"prompt"列を処理
test["response_a"] = test["response_a"].apply(process)  # テストデータの"response_a"列を処理
test["response_b"] = test["response_b"].apply(process)  # テストデータの"response_b"列を処理

print(f"train shape: {train.shape}")  # トレーニングデータの形状を表示
print(f"test shape: {test.shape}")  # テストデータの形状を表示
print("-"*90)
print(f"train missing values: {train.isnull().sum().sum()}")  # トレーニングデータの欠損値を表示
print(f"test missing values: {test.isnull().sum().sum()}")  # テストデータの欠損値を表示
print("-"*90)

train.head()  # トレーニングデータの先頭を表示
```

</div>
</details>

In [ ]:
train = pd.read_csv(config.train_path)  # トレーニングデータを読み込む
test = pd.read_csv(config.test_path)    # テストデータを読み込む
sample_submission = pd.read_csv(config.sample_submission_path)  # サンプル提出データを読み込む

if test.shape[0] < 10:  # テストデータのサンプル数が10未満の場合
    train = train.iloc[:10000]  # 最初の10000行のみをトレーニングデータに使用
    
def process(input_str):  # 入力文字列を処理する関数
    stripped_str = input_str.strip('[]')  # 角括弧を除去
    sentences = [s.strip('"') for s in stripped_str.split('","')]  # 行を分割し、不要なダブルクオートを除去
    return ' '.join(sentences)  # 処理した文を空白区切りで結合して返す

train["prompt"] = train["prompt"].apply(process)  # "prompt"列を処理
train["response_a"] = train["response_a"].apply(process)  # "response_a"列を処理
train["response_b"] = train["response_b"].apply(process)  # "response_b"列を処理

test["prompt"] = test["prompt"].apply(process)  # テストデータの"prompt"列を処理
test["response_a"] = test["response_a"].apply(process)  # テストデータの"response_a"列を処理
test["response_b"] = test["response_b"].apply(process)  # テストデータの"response_b"列を処理

print(f"train shape: {train.shape}")  # トレーニングデータの形状を表示
print(f"test shape: {test.shape}")  # テストデータの形状を表示
print("-"*90)
print(f"train missing values: {train.isnull().sum().sum()}")  # トレーニングデータの欠損値を表示
print(f"test missing values: {test.isnull().sum().sum()}")  # テストデータの欠損値を表示
print("-"*90)

train.head()  # トレーニングデータの先頭を表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
class Preprocessor:

    def cosine_sim(self, text1: str, text2: str):
        try:
            vectorizer = TfidfVectorizer(ngram_range=(1, 3))
            vectorizer.fit([text1, text2])
            output = vectorizer.transform([text1, text2]).toarray()
            cos_sim = cosine_similarity(output)
            return cos_sim[0][1]
        except:
            return np.nan

    def jaccard_sim(self, text1: str, text2: str):
        set1 = set(text1.split())
        set2 = set(text2.split())
        intersection = set1.intersection(set2)
        union = set1.union(set2)
        return len(intersection) / len(union)
    
    def count_new_lines(self, text: str) -> int:
        return text.count('\\n') 
    
    def count_quotes(self, text: str) -> int:
        single_quote_pattern = r"'(.*?)'"
        double_quote_pattern = r'"(.*?)"'
        single_quotes = re.findall(single_quote_pattern, text)
        double_quotes = re.findall(double_quote_pattern, text)
        total_quotes = len(single_quotes) + len(double_quotes)
        return len(single_quotes) + len(double_quotes)

    def tokenize(self, text: str):
        return nltk.word_tokenize(text.lower())

    def generate_ngrams(self, text: str, n: int):
        tokens = self.tokenize(text)
        return list(ngrams(tokens, n))

    def count_ngram_overlaps(self, text1: str, text2: str, n: int) -> int:
        try:
            ngrams1 = self.generate_ngrams(text1, n)
            ngrams2 = self.generate_ngrams(text2, n)
            counter1 = Counter(ngrams1)
            counter2 = Counter(ngrams2)
            overlap = counter1 & counter2
            overlap_count = sum(overlap.values())
            return overlap_count
        except:
            return 0
        
    def run(self, data: pd.DataFrame) -> pd.DataFrame:
        
        data["respa_respb_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 1), axis=1)
        data["respa_respb_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 2), axis=1)
        data["respa_respb_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 3), axis=1)

        data["respa_prompt_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 1), axis=1)
        data["respa_prompt_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 2), axis=1)
        data["respa_prompt_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 3), axis=1)

        data["respb_prompt_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 1), axis=1)
        data["respb_prompt_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 2), axis=1)
        data["respb_prompt_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 3), axis=1)
        
        data["respa_len"] = data["response_a"].apply(lambda x: len(self.tokenize(x)))
        data["respb_len"] = data["response_b"].apply(lambda x: len(self.tokenize(x)))
        data["prompt_len"] = data["prompt"].apply(lambda x: len(self.tokenize(x)))
        
        data["respa_new_lines"] = data["response_a"].apply(lambda x: self.count_new_lines(x))
        data["respb_new_lines"] = data["response_b"].apply(lambda x: self.count_new_lines(x))
        data["prompt_new_lines"] = data["prompt"].apply(lambda x: self.count_new_lines(x))
        
        data["respa_prompt_len_ratio"] = data["respa_len"] / data["prompt_len"]
        data["respb_prompt_len_ratio"] = data["respb_len"] / data["prompt_len"]
        data["respa_respb_len_ratio"] = data["respa_len"] / data["respb_len"]
        
        data["respa_respb_len_diff"] = data["respa_len"] - data["respb_len"]
        data["respa_prompt_len_diff"] = data["respa_len"] - data["prompt_len"]
        data["respb_prompt_len_diff"] = data["respb_len"] - data["prompt_len"]
        
        data["respa_prompt_overlap_unigram_len_ratio"] = data["respa_prompt_overlap_unigram"] / data["prompt_len"]
        data["respa_prompt_overlap_bigram_len_ratio"] = data["respa_prompt_overlap_bigram"] / data["prompt_len"]
        data["respa_prompt_overlap_trigram_len_ratio"] = data["respa_prompt_overlap_trigram"] / data["prompt_len"]

        data["respb_prompt_overlap_unigram_len_ratio"] = data["respb_prompt_overlap_unigram"] / data["prompt_len"]
        data["respb_prompt_overlap_bigram_len_ratio"] = data["respb_prompt_overlap_bigram"] / data["prompt_len"]
        data["respb_prompt_overlap_trigram_len_ratio"] = data["respb_prompt_overlap_trigram"] / data["prompt_len"]
        
        data["overlap_unigram_diff"] = data["respa_prompt_overlap_unigram"] - data["respb_prompt_overlap_unigram"]
        data["overlap_bigram_diff"] = data["respa_prompt_overlap_bigram"] - data["respb_prompt_overlap_bigram"]
        data["overlap_trigram_diff"] = data["respa_prompt_overlap_trigram"] - data["respb_prompt_overlap_trigram"]
        
        data["overlap_unigram_ratio"] = data["respb_prompt_overlap_unigram"] / data["respa_prompt_overlap_unigram"] 
        data["overlap_bigram_ratio"] = data["respb_prompt_overlap_bigram"] / data["respa_prompt_overlap_bigram"] 
        data["overlap_trigram_ratio"] = data["respb_prompt_overlap_trigram"] / data["respa_prompt_overlap_trigram"] 
        
        data["respa_quotes"] = data["response_a"].apply(lambda x: self.count_quotes(x))
        data["respb_quotes"] = data["response_b"].apply(lambda x: self.count_quotes(x))
        data["prompt_quotes"] = data["prompt"].apply(lambda x: self.count_quotes(x))
        
        data["respa_respb_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_a"], x["response_b"]), axis=1)
        data["respa_respb_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_a"], x["response_b"]), axis=1)
        
        data["respa_prompt_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_a"], x["prompt"]), axis=1)
        data["respa_prompt_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_a"], x["prompt"]), axis=1)
        
        data["respb_prompt_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_b"], x["prompt"]), axis=1)
        data["respb_prompt_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_b"], x["prompt"]), axis=1)
        
        data["jaccard_sim_diff"] = data["respa_prompt_jaccard_sim"] - data["respb_prompt_jaccard_sim"]
        data["jaccard_sim_ratio"] = data["respb_prompt_jaccard_sim"] / data["respa_prompt_jaccard_sim"]
        
        return data
```

</div>
<div class="column-right">

# 日本語訳

```python
class Preprocessor:

    def cosine_sim(self, text1: str, text2: str):  # コサイン類似度を計算するメソッド
        try:
            vectorizer = TfidfVectorizer(ngram_range=(1, 3))  # TF-IDFベクトル化器を作成（1～3グラム）
            vectorizer.fit([text1, text2])  # テキストをフィッティング
            output = vectorizer.transform([text1, text2]).toarray()  # TF-IDF行列を生成
            cos_sim = cosine_similarity(output)  # コサイン類似度を計算
            return cos_sim[0][1]  # 二つのテキスト間のコサイン類似度を返す
        except:
            return np.nan  # エラーが発生した場合はNaNを返す

    def jaccard_sim(self, text1: str, text2: str):  # ジャッカード類似度を計算するメソッド
        set1 = set(text1.split())  # text1を単語に分割しセットに変換
        set2 = set(text2.split())  # text2を単語に分割しセットに変換
        intersection = set1.intersection(set2)  # 二つのセットの積集合を計算
        union = set1.union(set2)  # 二つのセットの和集合を計算
        return len(intersection) / len(union)  # ジャッカード類似度を計算して返す
    
    def count_new_lines(self, text: str) -> int:  # テキスト内の改行数をカウントするメソッド
        return text.count('\\n')  # 改行の数を返す
    
    def count_quotes(self, text: str) -> int:  # テキスト内の引用符の数をカウントするメソッド
        single_quote_pattern = r"'(.*?)'"  # シングルクォートのパターン
        double_quote_pattern = r'"(.*?)"'  # ダブルクォートのパターン
        single_quotes = re.findall(single_quote_pattern, text)  # シングルクォートを探す
        double_quotes = re.findall(double_quote_pattern, text)  # ダブルクォートを探す
        total_quotes = len(single_quotes) + len(double_quotes)  # 合計の引用符の数をカウント
        return len(single_quotes) + len(double_quotes)  # 合計の引用符の数を返す

    def tokenize(self, text: str):  # テキストをトークン化するメソッド
        return nltk.word_tokenize(text.lower())  # 小文字に変換してトークン化して返す

    def generate_ngrams(self, text: str, n: int):  # n-グラムを生成するメソッド
        tokens = self.tokenize(text)  # テキストをトークン化
        return list(ngrams(tokens, n))  # n-グラムのリストを生成して返す

    def count_ngram_overlaps(self, text1: str, text2: str, n: int) -> int:  # n-グラムの重複数をカウントするメソッド
        try:
            ngrams1 = self.generate_ngrams(text1, n)  # text1からn-グラムを生成
            ngrams2 = self.generate_ngrams(text2, n)  # text2からn-グラムを生成
            counter1 = Counter(ngrams1)  # text1のn-グラムをカウント
            counter2 = Counter(ngrams2)  # text2のn-グラムをカウント
            overlap = counter1 & counter2  # カウンターの共通部分を計算
            overlap_count = sum(overlap.values())  # 重複の合計数を計算
            return overlap_count  # 重複の数を返す
        except:
            return 0  # エラーが発生した場合は0を返す
        
    def run(self, data: pd.DataFrame) -> pd.DataFrame:  # データを処理するメソッド
        data["respa_respb_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 1), axis=1)  # response_aとresponse_bのユニグラム重複を計算
        data["respa_respb_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 2), axis=1)  # response_aとresponse_bのバイグラム重複を計算
        data["respa_respb_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 3), axis=1)  # response_aとresponse_bのトライグラム重複を計算

        data["respa_prompt_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 1), axis=1)  # response_aとpromptのユニグラム重複を計算
        data["respa_prompt_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 2), axis=1)  # response_aとpromptのバイグラム重複を計算
        data["respa_prompt_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 3), axis=1)  # response_aとpromptのトライグラム重複を計算

        data["respb_prompt_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 1), axis=1)  # response_bとpromptのユニグラム重複を計算
        data["respb_prompt_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 2), axis=1)  # response_bとpromptのバイグラム重複を計算
        data["respb_prompt_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 3), axis=1)  # response_bとpromptのトライグラム重複を計算
        
        data["respa_len"] = data["response_a"].apply(lambda x: len(self.tokenize(x)))  # response_aのトークン数を計算
        data["respb_len"] = data["response_b"].apply(lambda x: len(self.tokenize(x)))  # response_bのトークン数を計算
        data["prompt_len"] = data["prompt"].apply(lambda x: len(self.tokenize(x)))  # promptのトークン数を計算
        
        data["respa_new_lines"] = data["response_a"].apply(lambda x: self.count_new_lines(x))  # response_aの改行数を数える
        data["respb_new_lines"] = data["response_b"].apply(lambda x: self.count_new_lines(x))  # response_bの改行数を数える
        data["prompt_new_lines"] = data["prompt"].apply(lambda x: self.count_new_lines(x))  # promptの改行数を数える
        
        data["respa_prompt_len_ratio"] = data["respa_len"] / data["prompt_len"]  # response_aの長さとpromptの比率を計算
        data["respb_prompt_len_ratio"] = data["respb_len"] / data["prompt_len"]  # response_bの長さとpromptの比率を計算
        data["respa_respb_len_ratio"] = data["respa_len"] / data["respb_len"]  # response_aとresponse_bの長さの比率を計算
        
        data["respa_respb_len_diff"] = data["respa_len"] - data["respb_len"]  # response_aとresponse_bの長さの差を計算
        data["respa_prompt_len_diff"] = data["respa_len"] - data["prompt_len"]  # response_aとpromptの長さの差を計算
        data["respb_prompt_len_diff"] = data["respb_len"] - data["prompt_len"]  # response_bとpromptの長さの差を計算
        
        data["respa_prompt_overlap_unigram_len_ratio"] = data["respa_prompt_overlap_unigram"] / data["prompt_len"]  # ユニグラム重複とpromptの長さの比率を計算
        data["respa_prompt_overlap_bigram_len_ratio"] = data["respa_prompt_overlap_bigram"] / data["prompt_len"]  # バイグラム重複とpromptの長さの比率を計算
        data["respa_prompt_overlap_trigram_len_ratio"] = data["respa_prompt_overlap_trigram"] / data["prompt_len"]  # トライグラム重複とpromptの長さの比率を計算

        data["respb_prompt_overlap_unigram_len_ratio"] = data["respb_prompt_overlap_unigram"] / data["prompt_len"]  # ユニグラム重複とpromptの長さの比率を計算
        data["respb_prompt_overlap_bigram_len_ratio"] = data["respb_prompt_overlap_bigram"] / data["prompt_len"]  # バイグラム重複とpromptの長さの比率を計算
        data["respb_prompt_overlap_trigram_len_ratio"] = data["respb_prompt_overlap_trigram"] / data["prompt_len"]  # トライグラム重複とpromptの長さの比率を計算
        
        data["overlap_unigram_diff"] = data["respa_prompt_overlap_unigram"] - data["respb_prompt_overlap_unigram"]  # ユニグラム重複の差を計算
        data["overlap_bigram_diff"] = data["respa_prompt_overlap_bigram"] - data["respb_prompt_overlap_bigram"]  # バイグラム重複の差を計算
        data["overlap_trigram_diff"] = data["respa_prompt_overlap_trigram"] - data["respb_prompt_overlap_trigram"]  # トライグラム重複の差を計算
        
        data["overlap_unigram_ratio"] = data["respb_prompt_overlap_unigram"] / data["respa_prompt_overlap_unigram"]  # ユニグラム重複の比率を計算
        data["overlap_bigram_ratio"] = data["respb_prompt_overlap_bigram"] / data["respa_prompt_overlap_bigram"]  # バイグラム重複の比率を計算
        data["overlap_trigram_ratio"] = data["respb_prompt_overlap_trigram"] / data["respa_prompt_overlap_trigram"]  # トライグラム重複の比率を計算 
        
        data["respa_quotes"] = data["response_a"].apply(lambda x: self.count_quotes(x))  # response_aの引用符の数をカウント
        data["respb_quotes"] = data["response_b"].apply(lambda x: self.count_quotes(x))  # response_bの引用符の数をカウント
        data["prompt_quotes"] = data["prompt"].apply(lambda x: self.count_quotes(x))  # promptの引用符の数をカウント
        
        data["respa_respb_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_a"], x["response_b"]), axis=1)  # response_aとresponse_bのコサイン類似度を計算
        data["respa_respb_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_a"], x["response_b"]), axis=1)  # response_aとresponse_bのジャッカード類似度を計算
        
        data["respa_prompt_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_a"], x["prompt"]), axis=1)  # response_aとpromptのコサイン類似度を計算
        data["respa_prompt_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_a"], x["prompt"]), axis=1)  # response_aとpromptのジャッカード類似度を計算
        
        data["respb_prompt_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_b"], x["prompt"]), axis=1)  # response_bとpromptのコサイン類似度を計算
        data["respb_prompt_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_b"], x["prompt"]), axis=1)  # response_bとpromptのジャッカード類似度を計算
        
        data["jaccard_sim_diff"] = data["respa_prompt_jaccard_sim"] - data["respb_prompt_jaccard_sim"]  # ジャッカード類似度の差を計算
        data["jaccard_sim_ratio"] = data["respb_prompt_jaccard_sim"] / data["respa_prompt_jaccard_sim"]  # ジャッカード類似度の比率を計算
        
        return data  # 処理後のデータを返す
```

</div>
</details>

In [ ]:
class Preprocessor:

    def cosine_sim(self, text1: str, text2: str):  # コサイン類似度を計算するメソッド
        try:
            vectorizer = TfidfVectorizer(ngram_range=(1, 3))  # TF-IDFベクトル化器を作成（1～3グラム）
            vectorizer.fit([text1, text2])  # テキストをフィッティング
            output = vectorizer.transform([text1, text2]).toarray()  # TF-IDF行列を生成
            cos_sim = cosine_similarity(output)  # コサイン類似度を計算
            return cos_sim[0][1]  # 二つのテキスト間のコサイン類似度を返す
        except:
            return np.nan  # エラーが発生した場合はNaNを返す

    def jaccard_sim(self, text1: str, text2: str):  # ジャッカード類似度を計算するメソッド
        set1 = set(text1.split())  # text1を単語に分割しセットに変換
        set2 = set(text2.split())  # text2を単語に分割しセットに変換
        intersection = set1.intersection(set2)  # 二つのセットの積集合を計算
        union = set1.union(set2)  # 二つのセットの和集合を計算
        return len(intersection) / len(union)  # ジャッカード類似度を計算して返す
    
    def count_new_lines(self, text: str) -> int:  # テキスト内の改行数をカウントするメソッド
        return text.count('\\n')  # 改行の数を返す
    
    def count_quotes(self, text: str) -> int:  # テキスト内の引用符の数をカウントするメソッド
        single_quote_pattern = r"'(.*?)'"  # シングルクォートのパターン
        double_quote_pattern = r'"(.*?)"'  # ダブルクォートのパターン
        single_quotes = re.findall(single_quote_pattern, text)  # シングルクォートを探す
        double_quotes = re.findall(double_quote_pattern, text)  # ダブルクォートを探す
        total_quotes = len(single_quotes) + len(double_quotes)  # 合計の引用符の数をカウント
        return len(single_quotes) + len(double_quotes)  # 合計の引用符の数を返す

    def tokenize(self, text: str):  # テキストをトークン化するメソッド
        return nltk.word_tokenize(text.lower())  # 小文字に変換してトークン化して返す

    def generate_ngrams(self, text: str, n: int):  # n-グラムを生成するメソッド
        tokens = self.tokenize(text)  # テキストをトークン化
        return list(ngrams(tokens, n))  # n-グラムのリストを生成して返す

    def count_ngram_overlaps(self, text1: str, text2: str, n: int) -> int:  # n-グラムの重複数をカウントするメソッド
        try:
            ngrams1 = self.generate_ngrams(text1, n)  # text1からn-グラムを生成
            ngrams2 = self.generate_ngrams(text2, n)  # text2からn-グラムを生成
            counter1 = Counter(ngrams1)  # text1のn-グラムをカウント
            counter2 = Counter(ngrams2)  # text2のn-グラムをカウント
            overlap = counter1 & counter2  # カウンターの共通部分を計算
            overlap_count = sum(overlap.values())  # 重複の合計数を計算
            return overlap_count  # 重複の数を返す
        except:
            return 0  # エラーが発生した場合は0を返す
        
    def run(self, data: pd.DataFrame) -> pd.DataFrame:  # データを処理するメソッド
        data["respa_respb_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 1), axis=1)  # response_aとresponse_bのユニグラム重複を計算
        data["respa_respb_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 2), axis=1)  # response_aとresponse_bのバイグラム重複を計算
        data["respa_respb_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 3), axis=1)  # response_aとresponse_bのトライグラム重複を計算

        data["respa_prompt_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 1), axis=1)  # response_aとpromptのユニグラム重複を計算
        data["respa_prompt_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 2), axis=1)  # response_aとpromptのバイグラム重複を計算
        data["respa_prompt_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 3), axis=1)  # response_aとpromptのトライグラム重複を計算

        data["respb_prompt_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 1), axis=1)  # response_bとpromptのユニグラム重複を計算
        data["respb_prompt_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 2), axis=1)  # response_bとpromptのバイグラム重複を計算
        data["respb_prompt_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 3), axis=1)  # response_bとpromptのトライグラム重複を計算
        
        data["respa_len"] = data["response_a"].apply(lambda x: len(self.tokenize(x)))  # response_aのトークン数を計算
        data["respb_len"] = data["response_b"].apply(lambda x: len(self.tokenize(x)))  # response_bのトークン数を計算
        data["prompt_len"] = data["prompt"].apply(lambda x: len(self.tokenize(x)))  # promptのトークン数を計算
        
        data["respa_new_lines"] = data["response_a"].apply(lambda x: self.count_new_lines(x))  # response_aの改行数を数える
        data["respb_new_lines"] = data["response_b"].apply(lambda x: self.count_new_lines(x))  # response_bの改行数を数える
        data["prompt_new_lines"] = data["prompt"].apply(lambda x: self.count_new_lines(x))  # promptの改行数を数える
        
        data["respa_prompt_len_ratio"] = data["respa_len"] / data["prompt_len"]  # response_aの長さとpromptの比率を計算
        data["respb_prompt_len_ratio"] = data["respb_len"] / data["prompt_len"]  # response_bの長さとpromptの比率を計算
        data["respa_respb_len_ratio"] = data["respa_len"] / data["respb_len"]  # response_aとresponse_bの長さの比率を計算
        
        data["respa_respb_len_diff"] = data["respa_len"] - data["respb_len"]  # response_aとresponse_bの長さの差を計算
        data["respa_prompt_len_diff"] = data["respa_len"] - data["prompt_len"]  # response_aとpromptの長さの差を計算
        data["respb_prompt_len_diff"] = data["respb_len"] - data["prompt_len"]  # response_bとpromptの長さの差を計算
        
        data["respa_prompt_overlap_unigram_len_ratio"] = data["respa_prompt_overlap_unigram"] / data["prompt_len"]  # ユニグラム重複とpromptの長さの比率を計算
        data["respa_prompt_overlap_bigram_len_ratio"] = data["respa_prompt_overlap_bigram"] / data["prompt_len"]  # バイグラム重複とpromptの長さの比率を計算
        data["respa_prompt_overlap_trigram_len_ratio"] = data["respa_prompt_overlap_trigram"] / data["prompt_len"]  # トライグラム重複とpromptの長さの比率を計算

        data["respb_prompt_overlap_unigram_len_ratio"] = data["respb_prompt_overlap_unigram"] / data["prompt_len"]  # ユニグラム重複とpromptの長さの比率を計算
        data["respb_prompt_overlap_bigram_len_ratio"] = data["respb_prompt_overlap_bigram"] / data["prompt_len"]  # バイグラム重複とpromptの長さの比率を計算
        data["respb_prompt_overlap_trigram_len_ratio"] = data["respb_prompt_overlap_trigram"] / data["prompt_len"]  # トライグラム重複とpromptの長さの比率を計算
        
        data["overlap_unigram_diff"] = data["respa_prompt_overlap_unigram"] - data["respb_prompt_overlap_unigram"]  # ユニグラム重複の差を計算
        data["overlap_bigram_diff"] = data["respa_prompt_overlap_bigram"] - data["respb_prompt_overlap_bigram"]  # バイグラム重複の差を計算
        data["overlap_trigram_diff"] = data["respa_prompt_overlap_trigram"] - data["respb_prompt_overlap_trigram"]  # トライグラム重複の差を計算
        
        data["overlap_unigram_ratio"] = data["respb_prompt_overlap_unigram"] / data["respa_prompt_overlap_unigram"]  # ユニグラム重複の比率を計算
        data["overlap_bigram_ratio"] = data["respb_prompt_overlap_bigram"] / data["respa_prompt_overlap_bigram"]  # バイグラム重複の比率を計算
        data["overlap_trigram_ratio"] = data["respb_prompt_overlap_trigram"] / data["respa_prompt_overlap_trigram"]  # トライグラム重複の比率を計算 
        
        data["respa_quotes"] = data["response_a"].apply(lambda x: self.count_quotes(x))  # response_aの引用符の数をカウント
        data["respb_quotes"] = data["response_b"].apply(lambda x: self.count_quotes(x))  # response_bの引用符の数をカウント
        data["prompt_quotes"] = data["prompt"].apply(lambda x: self.count_quotes(x))  # promptの引用符の数をカウント
        
        data["respa_respb_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_a"], x["response_b"]), axis=1)  # response_aとresponse_bのコサイン類似度を計算
        data["respa_respb_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_a"], x["response_b"]), axis=1)  # response_aとresponse_bのジャッカード類似度を計算
        
        data["respa_prompt_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_a"], x["prompt"]), axis=1)  # response_aとpromptのコサイン類似度を計算
        data["respa_prompt_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_a"], x["prompt"]), axis=1)  # response_aとpromptのジャッカード類似度を計算
        
        data["respb_prompt_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_b"], x["prompt"]), axis=1)  # response_bとpromptのコサイン類似度を計算
        data["respb_prompt_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_b"], x["prompt"]), axis=1)  # response_bとpromptのジャッカード類似度を計算
        
        data["jaccard_sim_diff"] = data["respa_prompt_jaccard_sim"] - data["respb_prompt_jaccard_sim"]  # ジャッカード類似度の差を計算
        data["jaccard_sim_ratio"] = data["respb_prompt_jaccard_sim"] / data["respa_prompt_jaccard_sim"]  # ジャッカード類似度の比率を計算
        
        return data  # 処理後のデータを返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
%%time
preprocessor = Preprocessor()
train = preprocessor.run(train)
test = preprocessor.run(test)
train.head()
```

</div>
<div class="column-right">

# 日本語訳

```python
%%time
preprocessor = Preprocessor()  # Preprocessorのインスタンスを生成
train = preprocessor.run(train)  # トレーニングデータに対して前処理を実行
test = preprocessor.run(test)  # テストデータに対して前処理を実行
train.head()  # トレーニングデータの先頭を表示
```

</div>
</details>

In [ ]:
%%time
preprocessor = Preprocessor()  # Preprocessorのインスタンスを生成
train = preprocessor.run(train)  # トレーニングデータに対して前処理を実行
test = preprocessor.run(test)  # テストデータに対して前処理を実行
train.head()  # トレーニングデータの先頭を表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
drop_cols = ["id", "response_a", "response_b", "prompt"]
target_cols = ["winner_model_a", "winner_model_b", "winner_tie"]
target = "target"

train[target] = np.nan
for idx, t in enumerate(target_cols):
    train.loc[train[t] == 1, target] = idx
train[target] = train[target].astype("int32")
    
train.head()
```

</div>
<div class="column-right">

# 日本語訳

```python
drop_cols = ["id", "response_a", "response_b", "prompt"]  # 除外する列のリスト
target_cols = ["winner_model_a", "winner_model_b", "winner_tie"]  # ターゲットとなる列のリスト
target = "target"  # ターゲットの名前

train[target] = np.nan  # ターゲット列をNaNで初期化
for idx, t in enumerate(target_cols):  # ターゲット列をループ
    train.loc[train[t] == 1, target] = idx  # 勝者モデルをターゲットとして設定
train[target] = train[target].astype("int32")  # ターゲット列の型を整数型に変換
    
train.head()  # トレーニングデータの先頭を表示
```

</div>
</details>

In [ ]:
drop_cols = ["id", "response_a", "response_b", "prompt"]  # 除外する列のリスト
target_cols = ["winner_model_a", "winner_model_b", "winner_tie"]  # ターゲットとなる列のリスト
target = "target"  # ターゲットの名前

train[target] = np.nan  # ターゲット列をNaNで初期化
for idx, t in enumerate(target_cols):  # ターゲット列をループ
    train.loc[train[t] == 1, target] = idx  # 勝者モデルをターゲットとして設定
train[target] = train[target].astype("int32")  # ターゲット列の型を整数型に変換
    
train.head()  # トレーニングデータの先頭を表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
X = train.drop(columns=target_cols+drop_cols+[target]+["model_a", "model_b"], axis=1)
y = train[target]
X_test = test.drop(columns=drop_cols, axis=1)

X = X.replace([-np.inf, np.inf], np.nan)
X_test = X_test.replace([-np.inf, np.inf], np.nan)
```

</div>
<div class="column-right">

# 日本語訳

```python
X = train.drop(columns=target_cols + drop_cols + [target] + ["model_a", "model_b"], axis=1)  # 特徴量Xを定義（不要な列を除外）
y = train[target]  # ターゲットyを定義
X_test = test.drop(columns=drop_cols, axis=1)  # テストデータから特徴量X_testを定義

X = X.replace([-np.inf, np.inf], np.nan)  # 特徴量Xの無限値をNaNに置き換え
X_test = X_test.replace([-np.inf, np.inf], np.nan)  # テストデータの無限値をNaNに置き換え
```

</div>
</details>

In [ ]:
X = train.drop(columns=target_cols + drop_cols + [target] + ["model_a", "model_b"], axis=1)  # 特徴量Xを定義（不要な列を除外）
y = train[target]  # ターゲットyを定義
X_test = test.drop(columns=drop_cols, axis=1)  # テストデータから特徴量X_testを定義

X = X.replace([-np.inf, np.inf], np.nan)  # 特徴量Xの無限値をNaNに置き換え
X_test = X_test.replace([-np.inf, np.inf], np.nan)  # テストデータの無限値をNaNに置き換え

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
cv = StratifiedKFold(n_splits=config.n_splits, shuffle=True, random_state=config.seed)
test_preds = np.zeros(shape=(X_test.shape[0], y.nunique()))
cv_scores = list()

features = X.columns.tolist()
feat_imp_df = pd.DataFrame({"feature": features})

for idx, (train_idx, val_idx) in enumerate(cv.split(X, y)):
    print(f"| Fold {idx+1} |".center(90, "="))
    X_train, y_train = X.loc[train_idx], y.loc[train_idx]
    X_val, y_val = X.loc[val_idx], y.loc[val_idx]

    print(f'train: {X_train.shape}')
    print(f'val: {X_val.shape}')
    
    model = xgb.XGBClassifier(
        objective='multi:softprob',
        num_class=3,
        eval_metric='mlogloss',
        subsample=0.8,
        n_estimators=650,
        learning_rate=0.045,
        max_depth=5,
        random_state=config.seed
    )
    
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        early_stopping_rounds=75,
        verbose=75
    )
    
    val_preds = model.predict_proba(X_val)
    val_log_loss = log_loss(y_val, val_preds, eps="auto")
    print(f"val log loss: {val_log_loss:.5f}")
    cv_scores.append(val_log_loss)
    
    test_preds += model.predict_proba(X_test) / cv.get_n_splits()
    
    feat_imp_df = feat_imp_df.merge(
        pd.DataFrame(
            {
                "feature": features,
                f"fold_{idx+1}_feat_imp": model.feature_importances_,
            }
        ),
        on=["feature"],
        how="left",
    )

print("="*90)
print(f"CV: {np.mean(cv_scores):.5f}")

feat_imp_df["avg_importance"] = feat_imp_df.iloc[:, 1:].mean(axis=1)
plt.figure(figsize=(12, 10))
sns.barplot(
    data=feat_imp_df.sort_values(by="avg_importance", ascending=False).iloc[
        :50
    ],
    x="avg_importance",
    y="feature",
    color="royalblue",
    width=0.75,
)
plt.title("Average Feature Importances of All Folds", size=12)
plt.show()
```

</div>
<div class="column-right">

# 日本語訳

```python
cv = StratifiedKFold(n_splits=config.n_splits, shuffle=True, random_state=config.seed)  # ストラティファイドKフォールドの設定
test_preds = np.zeros(shape=(X_test.shape[0], y.nunique()))  # テスト予測の初期化
cv_scores = list()  # クロスバリデーションスコアのリスト

features = X.columns.tolist()  # 特徴量のリストを取得
feat_imp_df = pd.DataFrame({"feature": features})  # 特徴量の重要度を格納するデータフレームを作成

for idx, (train_idx, val_idx) in enumerate(cv.split(X, y)):  # Kフォールド分割に基づいて訓練インデックスと検証インデックスを取得
    print(f"| Fold {idx+1} |".center(90, "="))  # 現在のフォールドの表示
    X_train, y_train = X.loc[train_idx], y.loc[train_idx]  # 訓練データとラベルの取得
    X_val, y_val = X.loc[val_idx], y.loc[val_idx]  # 検証データとラベルの取得

    print(f'train: {X_train.shape}')  # 訓練データの形状を表示
    print(f'val: {X_val.shape}')  # 検証データの形状を表示
    
    model = xgb.XGBClassifier(  # XGBoostの分類器を定義
        objective='multi:softprob',  # 目的関数を多クラスの確率に設定
        num_class=3,  # クラス数を3に設定
        eval_metric='mlogloss',  # 評価指標を対数損失に設定
        subsample=0.8,  # サンプリング率を0.8に設定
        n_estimators=650,  # 弱学習器の数を650に設定
        learning_rate=0.045,  # 学習率を0.045に設定
        max_depth=5,  # 木の最大深さを5に設定
        random_state=config.seed  # 乱数シードを設定
    )
    
    model.fit(  # モデルの訓練
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],  # 評価セットを訓練データと検証データに設定
        early_stopping_rounds=75,  # 75ラウンドで早期停止
        verbose=75  # 75回ごとに出力
    )
    
    val_preds = model.predict_proba(X_val)  # 検証データに対する確率予測
    val_log_loss = log_loss(y_val, val_preds, eps="auto")  # 検証データに対する対数損失を計算
    print(f"val log loss: {val_log_loss:.5f}")  # 検証データの対数損失を表示
    cv_scores.append(val_log_loss)  # クロスバリデーションスコアを追加
    
    test_preds += model.predict_proba(X_test) / cv.get_n_splits()  # 各フォールドのテスト予測を累積
    
    feat_imp_df = feat_imp_df.merge(  # 特徴量の重要度をデータフレームに追加
        pd.DataFrame(
            {
                "feature": features,  # 特徴量名
                f"fold_{idx+1}_feat_imp": model.feature_importances_,  # 各フォールドの特徴量重要度
            }
        ),
        on=["feature"],
        how="left",
    )

print("="*90)  # 区切り線を表示
print(f"CV: {np.mean(cv_scores):.5f}")  # クロスバリデーションの平均スコアを表示

feat_imp_df["avg_importance"] = feat_imp_df.iloc[:, 1:].mean(axis=1)  # 各特徴量の平均重要度を計算
plt.figure(figsize=(12, 10))  # プロットサイズを設定
sns.barplot(  # バープロットを作成
    data=feat_imp_df.sort_values(by="avg_importance", ascending=False).iloc[:50],  # 重要度が高い50の特徴量を表示
    x="avg_importance",
    y="feature",
    color="royalblue",
    width=0.75,
)
plt.title("Average Feature Importances of All Folds", size=12)  # タイトルを設定
plt.show()  # プロットを表示
```

</div>
</details>

In [ ]:
cv = StratifiedKFold(n_splits=config.n_splits, shuffle=True, random_state=config.seed)  # ストラティファイドKフォールドの設定
test_preds = np.zeros(shape=(X_test.shape[0], y.nunique()))  # テスト予測の初期化
cv_scores = list()  # クロスバリデーションスコアのリスト

features = X.columns.tolist()  # 特徴量のリストを取得
feat_imp_df = pd.DataFrame({"feature": features})  # 特徴量の重要度を格納するデータフレームを作成

for idx, (train_idx, val_idx) in enumerate(cv.split(X, y)):  # Kフォールド分割に基づいて訓練インデックスと検証インデックスを取得
    print(f"| Fold {idx+1} |".center(90, "="))  # 現在のフォールドの表示
    X_train, y_train = X.loc[train_idx], y.loc[train_idx]  # 訓練データとラベルの取得
    X_val, y_val = X.loc[val_idx], y.loc[val_idx]  # 検証データとラベルの取得

    print(f'train: {X_train.shape}')  # 訓練データの形状を表示
    print(f'val: {X_val.shape}')  # 検証データの形状を表示
    
    model = xgb.XGBClassifier(  # XGBoostの分類器を定義
        objective='multi:softprob',  # 目的関数を多クラスの確率に設定
        num_class=3,  # クラス数を3に設定
        eval_metric='mlogloss',  # 評価指標を対数損失に設定
        subsample=0.8,  # サンプリング率を0.8に設定
        n_estimators=650,  # 弱学習器の数を650に設定
        learning_rate=0.045,  # 学習率を0.045に設定
        max_depth=5,  # 木の最大深さを5に設定
        random_state=config.seed  # 乱数シードを設定
    )
    
    model.fit(  # モデルの訓練
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],  # 評価セットを訓練データと検証データに設定
        early_stopping_rounds=75,  # 75ラウンドで早期停止
        verbose=75  # 75回ごとに出力
    )
    
    val_preds = model.predict_proba(X_val)  # 検証データに対する確率予測
    val_log_loss = log_loss(y_val, val_preds, eps="auto")  # 検証データに対する対数損失を計算
    print(f"val log loss: {val_log_loss:.5f}")  # 検証データの対数損失を表示
    cv_scores.append(val_log_loss)  # クロスバリデーションスコアを追加
    
    test_preds += model.predict_proba(X_test) / cv.get_n_splits()  # 各フォールドのテスト予測を累積
    
    feat_imp_df = feat_imp_df.merge(  # 特徴量の重要度をデータフレームに追加
        pd.DataFrame(
            {
                "feature": features,  # 特徴量名
                f"fold_{idx+1}_feat_imp": model.feature_importances_,  # 各フォールドの特徴量重要度
            }
        ),
        on=["feature"],
        how="left",
    )

print("="*90)  # 区切り線を表示
print(f"CV: {np.mean(cv_scores):.5f}")  # クロスバリデーションの平均スコアを表示

feat_imp_df["avg_importance"] = feat_imp_df.iloc[:, 1:].mean(axis=1)  # 各特徴量の平均重要度を計算
plt.figure(figsize=(12, 10))  # プロットサイズを設定
sns.barplot(  # バープロットを作成
    data=feat_imp_df.sort_values(by="avg_importance", ascending=False).iloc[:50],  # 重要度が高い50の特徴量を表示
    x="avg_importance",
    y="feature",
    color="royalblue",
    width=0.75,
)
plt.title("Average Feature Importances of All Folds", size=12)  # タイトルを設定
plt.show()  # プロットを表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
for idx, t in enumerate(target_cols):
    sample_submission[t] = test_preds[:, idx]
sample_submission.head()
```

</div>
<div class="column-right">

# 日本語訳

```python
for idx, t in enumerate(target_cols):  # ターゲット列をループ
    sample_submission[t] = test_preds[:, idx]  # ターゲット列にテスト予測を格納
sample_submission.head()  # サンプル提出データの先頭を表示
```

</div>
</details>

In [ ]:
for idx, t in enumerate(target_cols):  # ターゲット列をループ
    sample_submission[t] = test_preds[:, idx]  # ターゲット列にテスト予測を格納
sample_submission.head()  # サンプル提出データの先頭を表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
sample_submission.to_csv("submission.csv", index=False)
```

</div>
<div class="column-right">

# 日本語訳

```python
sample_submission.to_csv("submission.csv", index=False)  # 提出ファイルとしてCSVに保存
```

</div>
</details>

In [ ]:
sample_submission.to_csv("submission.csv", index=False)  # 提出ファイルとしてCSVに保存